In [ ]:
# import library
import numpy as py
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM , SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re

In [ ]:
# read and load data
data = pd.read_csv('/content/Sentiment.csv')


In [ ]:
#select features important
data = data[['text','sentiment']]

In [ ]:
### expcetion neutral case from lable sentiment
data = data[data.sentiment != 'Neutral']
#### lambda func:- create function without name
#### where x is parametar, x.lower is the return
####  data['text'].apply represent call by value to x
data['text']= data['text'].apply(lambda x: x.lower()) # transformation upper case to lower
data['text']= data['text'].apply(lambda x:re.sub(r'[^a-zA-Z0-9\s]','',x)) #Determine the things I want in the speech

<ipython-input-5-718aca211509>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text']= data['text'].apply(lambda x: x.lower()) # transformation upper case to lower
<ipython-input-5-718aca211509>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text']= data['text'].apply(lambda x:re.sub(r'[^a-zA-Z0-9\s]','',x)) #Determine the things I want in the speech


In [ ]:
for idx, row in data.iterrows():
  row[0] = row[0].replace('rt',' ') # replace rt with a space

In [ ]:
max_features=2000 # the variable is represent dictionary

## tokenizer is a class Responsible for dividing the sentence, as it divides every space
## create object from tokenizer class and give it two variables, one is the size of the dictionary and the other is the thing I want to divide from
tokenizer= Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
x = tokenizer.texts_to_sequences(data['text'].values)

x= pad_sequences(x) ## To make all sentences equal, where each sentence will be equal to the longest sentence in text label

In [ ]:
embed_dim =128 ## represent numbers of weights
lstm_out=196 ## represent numbers of units

model= Sequential()
#### Embedding is the first layer in the model Responsible for give Giving words that are close or that have a relationship between them similar values
### max_features --> dictionary
### embed_dim --> numbers of weights
## input_length --> It represents the input of the layer
model.add(Embedding(max_features, embed_dim,input_length=x.shape[1]))

### drop from output of the Embedding layers to  prevent happening overfitting
model.add(SpatialDropout1D(0.4))

### lstm layers tekes 3 variables
### lstm_out --> numbers of units
### dropout --> represent a random drop to the input
### recurrent_dropout--> represent a random drop to the recurrent
model.add(LSTM(lstm_out, dropout=0.2 , recurrent_dropout= 0.2))

model.add(Dense(2,'softmax'))
model.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

y=pd.get_dummies(data['sentiment']).values ## one hot incoding


In [ ]:
x_train, x_test ,y_train, y_test= train_test_split(x,y, test_size=0.2,random_state=32)


In [ ]:
model.fit(x_train, y_train, epochs=5 , batch_size=30)

Epoch 1/5
287/287 [==============================] - 54s 172ms/step - loss: 0.4207 - accuracy: 0.8237
Epoch 2/5
287/287 [==============================] - 51s 178ms/step - loss: 0.3151 - accuracy: 0.8646
Epoch 3/5
287/287 [==============================] - 53s 185ms/step - loss: 0.2759 - accuracy: 0.8849
Epoch 4/5
287/287 [==============================] - 49s 171ms/step - loss: 0.2504 - accuracy: 0.8928
Epoch 5/5
287/287 [==============================] - 49s 172ms/step - loss: 0.2279 - accuracy: 0.9061
